In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install numpy
%pip install tensorflow


In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Attention, Dense, GlobalAveragePooling1D, Input, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import string
import spacy
from datetime import datetime


In [8]:
df = pd.read_csv("../../../data/processed/train_preprocess_v1.csv")
df_test = pd.read_csv("../../../data/processed/test_preprocess_v1.csv")


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  8950 non-null   object
 1   label                               8950 non-null   int64 
 2   statement                           8950 non-null   object
 3   subject                             8950 non-null   object
 4   speaker                             8950 non-null   object
 5   speaker_job                         8950 non-null   object
 6   state_info                          8950 non-null   object
 7   party_affiliation                   8950 non-null   object
 8   party_affiliation_uni               8950 non-null   object
 9   party_affiliation_category_map      8950 non-null   object
 10  statement_tokens                    8950 non-null   object
 11  num_tokens                          8950 non-null   int6

## Intento 1

In [15]:


# Cargar los datos

# Ver las primeras filas para inspección
print(df.head())

# Preprocesamiento básico del texto (se puede mejorar luego)
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])  # Eliminar caracteres no alfanuméricos
    return text

# Aplicar preprocesamiento al texto
df['processed_statement'] = df['statement_tokens_without_stopwords'].apply(preprocess_text)

# Tokenización
max_vocab = 10000  # Número máximo de palabras en el vocabulario
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(df['processed_statement'])
X = tokenizer.texts_to_sequences(df['processed_statement'])

# Padding de secuencias
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de las secuencias
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

# Labels
y = df['label'].values

# Dividir en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo BiLSTM + Atención
input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))

# Capa de embedding
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_layer)

# Capa LSTM bidireccional
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)

# Capa de atención
attention_layer = Attention()([lstm_layer, lstm_layer])  # Aplicar atención entre la secuencia y sí misma

# Capa de pooling
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(pooling_layer)

# Definir el modelo completo
model = Model(inputs=input_layer, outputs=output_layer)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))


            id  label                                          statement  \
0  81f884c64a7      1  china is in the south china sea and (building)...   
1  30c2723a188      0  with the resources it takes to execute just ov...   
2  6936b216e5d      0  the (wisconsin) governor has proposed tax give...   
3  b5cd9195738      1  says her representation of an ex-boyfriend who...   
4  84f8dac7737      0  at protests in wisconsin against proposed coll...   

                                             subject  \
0                      china,foreign-policy,military   
1                                        health-care   
2           corporations,pundits,taxes,abc-news-week   
3  candidates-biography,children,ethics,families,...   
4                     health-care,labor,state-budget   

                      speaker            speaker_job        state_info  \
0                donald-trump        president-elect          new york   
1                  chris-dodd           u.s. senator      

c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 100, 100)  │  1,000,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ (None, 100, 128)  │     84,480 │ embedding_2[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_2         │ (None, 100, 128)  │          0 │ bidirectional_2[… │
│ (Attention)         │                   │            │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ attention_2[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        129 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,084,609 (4.14 MB)

 Trainable params: 1,084,609 (4.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
224/224 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.6405 - loss: 0.6595 - val_accuracy: 0.6587 - val_loss: 0.6438
Epoch 2/3
224/224 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.6497 - loss: 0.6479 - val_accuracy: 0.6587 - val_loss: 0.6411
Epoch 3/3
224/224 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.6611 - loss: 0.6239 - val_accuracy: 0.6341 - val_loss: 0.6345


In [17]:
import datetime
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Cargar el conjunto de datos de prueba df_test

# Preprocesamiento del texto en df_test (asegúrate de que el preprocesamiento sea el mismo que para df)
df_test['processed_statement'] = df_test['statement_tokens_without_stopwords'].apply(preprocess_text)

# Tokenización de las declaraciones en el conjunto de prueba
X_test_df = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias de prueba
X_test_df = pad_sequences(X_test_df, maxlen=MAX_SEQUENCE_LENGTH)

# Predicciones con el modelo
y_pred = model.predict(X_test_df)
y_pred = (y_pred > 0.5).astype(int)  # Convertir las probabilidades en etiquetas 0 o 1


# 2) Predicción y creación del CSV para envío
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que la predicción sea un vector plano
})

# Guardar el archivo CSV de envío
filename = f"submission_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())


120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
✅ Submission generada correctamente: 'submission_20250514_0007.csv'
            id  label
0  dc32e5ffa8b      1
1  aa49bb41cab      1
2  dddc8d12ac1      1
3  bcfe8f51667      1
4  eedbbaff5ab      1


## Intento 2 

In [6]:
nlp = spacy.load("en_core_web_sm")


In [11]:
# Función de preprocesamiento avanzada: lematización y eliminación de stopwords
def preprocess_text_advanced(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Procesar con spaCy para obtener el lematizador
    doc = nlp(text)
    # Lemmatización y eliminación de stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Aplicar el preprocesamiento avanzado al conjunto de datos
df['processed_statement'] = df['statement'].apply(preprocess_text_advanced)
df_test['processed_statement'] = df_test['statement'].apply(preprocess_text_advanced)

# Tokenización
max_vocab = 10000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(df['processed_statement'])

# Convertir los textos en secuencias de tokens
X_train = tokenizer.texts_to_sequences(df['processed_statement'])
X_test = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de las secuencias
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encoding de las columnas categóricas adicionales
subject_encoded = pd.get_dummies(df['processed_subject'])
speaker_encoded = pd.get_dummies(df['speaker'])
party_affiliation_encoded = pd.get_dummies(df['party_affiliation'])

# Unir las características codificadas con las secuencias de texto
X_additional_features = pd.concat([subject_encoded, speaker_encoded, party_affiliation_encoded], axis=1)

# Labels
y = df['label'].values

# Dividir el dataset en entrenamiento y validación (80-20)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# Definir las entradas para el modelo
input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
input_additional = Input(shape=(X_additional_features.shape[1],))  # Características adicionales

# Capa de embedding para el texto
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_text)

# Capa LSTM bidireccional con Dropout
lstm_layer = Bidirectional(LSTM(128, return_sequences=True))(embedding_layer)
dropout_layer = Dropout(0.5)(lstm_layer)

# Capa de atención
attention_layer = Attention()([dropout_layer, dropout_layer])

# Capa de pooling
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Concatenar las características adicionales
merged = Concatenate()([pooling_layer, input_additional])

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(merged)

# Definir el modelo
model = Model(inputs=[input_text, input_additional], outputs=output_layer)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit([X_train, X_additional_features.iloc[:len(X_train)]], y_train, epochs=5, batch_size=32, validation_data=([X_val, X_additional_features.iloc[:len(X_val)]], y_val))

# Predicción sobre el conjunto de prueba
y_pred_prob = model.predict([X_test, X_additional_features.iloc[:len(X_test)]])  # Predicción de probabilidades
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades en etiquetas binarias (0 o 1)



# Generación del CSV con las predicciones para df_test
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que 'y_pred' tenga la forma correcta
})

# Guardar el archivo de predicciones
filename = f"biLSTM_submission_{datetime.now().strftime('%Y%m%d')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())

c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 100, 100)  │  1,000,000 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ (None, 100, 256)  │    234,496 │ embedding_2[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 100, 256)  │          0 │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_2         │ (None, 100, 256)  │          0 │ dropout_2[0][0],  │
│ (Attention)         │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ attention_2[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 6067)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 6323)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │      6,324 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,240,820 (4.73 MB)

 Trainable params: 1,240,820 (4.73 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.6408 - loss: 0.6523 - val_accuracy: 0.6587 - val_loss: 0.6442
Epoch 2/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.6467 - loss: 0.6386 - val_accuracy: 0.6581 - val_loss: 0.6497
Epoch 3/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.7011 - loss: 0.5658 - val_accuracy: 0.5737 - val_loss: 0.6601
Epoch 4/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.7481 - loss: 0.5177 - val_accuracy: 0.5631 - val_loss: 0.7011
Epoch 5/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.7639 - loss: 0.4761 - val_accuracy: 0.5966 - val_loss: 0.7037
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
✅ Submission generada correctamente: 'biLSTM_submission_20250514.csv'
            id  label
0  dc32e5ffa8b      0
1  aa49bb41cab      1
2  dddc8d12ac1      1
3  bcfe8f51667      1
4  eedbbaff5ab      0


## Intento 3

In [14]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, GRU, Input, Dropout, GlobalAveragePooling1D, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime
import string
import spacy
from sklearn.utils.class_weight import compute_class_weight

# Cargar el modelo de spaCy
nlp = spacy.load("en_core_web_sm")

# Función de preprocesamiento avanzada: lematización y eliminación de stopwords
def preprocess_text_advanced(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Procesar con spaCy para obtener el lematizador
    doc = nlp(text)
    # Lemmatización y eliminación de stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Aplicar el preprocesamiento avanzado al conjunto de datos
df['processed_statement'] = df['statement'].apply(preprocess_text_advanced)
df_test['processed_statement'] = df_test['statement'].apply(preprocess_text_advanced)

# Tokenización
max_vocab = 10000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(df['processed_statement'])

# Convertir los textos en secuencias de tokens
X_train = tokenizer.texts_to_sequences(df['processed_statement'])
X_test = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de las secuencias
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encoding de las columnas categóricas adicionales
subject_encoded = pd.get_dummies(df['processed_subject'])
speaker_encoded = pd.get_dummies(df['speaker'])
party_affiliation_encoded = pd.get_dummies(df['party_affiliation'])

# Unir las características codificadas con las secuencias de texto
X_additional_features = pd.concat([subject_encoded, speaker_encoded, party_affiliation_encoded], axis=1)

# Labels
y = df['label'].values

# Dividir el dataset en entrenamiento y validación (80-20)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# Calcular los pesos de clase para balancear el dataset
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Definir las entradas para el modelo
input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
input_additional = Input(shape=(X_additional_features.shape[1],))  # Características adicionales

# Capa de embedding para el texto
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_text)

# Capa LSTM bidireccional con más unidades y Dropout
lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(embedding_layer)  # Más unidades LSTM
dropout_layer = Dropout(0.5)(lstm_layer)

# Capa de atención
attention_layer = Attention()([dropout_layer, dropout_layer])

# Capa de pooling
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Concatenar las características adicionales
merged = Concatenate()([pooling_layer, input_additional])

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(merged)

# Definir el modelo
model = Model(inputs=[input_text, input_additional], outputs=output_layer)

# Crear un callback para reducir la tasa de aprendizaje si el modelo no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo con el callback de ReduceLROnPlateau
model.fit([X_train, X_additional_features.iloc[:len(X_train)]], y_train, epochs=5, batch_size=32, 
          validation_data=([X_val, X_additional_features.iloc[:len(X_val)]], y_val), 
          class_weight={0: class_weights[0], 1: class_weights[1]},
          callbacks=[reduce_lr])

# Predicción sobre el conjunto de prueba
y_pred_prob = model.predict([X_test, X_additional_features.iloc[:len(X_test)]])  # Predicción de probabilidades
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades en etiquetas binarias (0 o 1)



# 2) Generar el CSV con las predicciones para df_test
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que 'y_pred' tenga la forma correcta
})

# Guardar el archivo de predicciones
filename = f"biLSTM_submission_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())


c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 100, 100)  │  1,000,000 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_4     │ (None, 100, 512)  │    731,136 │ embedding_4[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 100, 512)  │          0 │ bidirectional_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_4         │ (None, 100, 512)  │          0 │ dropout_4[0][0],  │
│ (Attention)         │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ attention_4[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 6067)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 6579)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │      6,580 │ concatenate_4[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,737,716 (6.63 MB)

 Trainable params: 1,737,716 (6.63 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 58s 245ms/step - accuracy: 0.4760 - loss: 0.6945 - val_accuracy: 0.6553 - val_loss: 0.6825 - learning_rate: 0.0010
Epoch 2/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 50s 225ms/step - accuracy: 0.6397 - loss: 0.6864 - val_accuracy: 0.5223 - val_loss: 0.6918 - learning_rate: 0.0010
Epoch 3/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 50s 221ms/step - accuracy: 0.7060 - loss: 0.6773 - val_accuracy: 0.3950 - val_loss: 0.7059 - learning_rate: 0.0010
Epoch 4/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 50s 222ms/step - accuracy: 0.7027 - loss: 0.6749 - val_accuracy: 0.5436 - val_loss: 0.6893 - learning_rate: 5.0000e-04
Epoch 5/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 51s 229ms/step - accuracy: 0.7146 - loss: 0.6676 - val_accuracy: 0.4989 - val_loss: 0.6941 - learning_rate: 5.0000e-04
120/120 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step
✅ Submission generada correctamente: 'biLSTM_submission_20250514_0243.csv'
            id  label
0  dc32e5ffa8b      1
1  aa49bb41cab      1
2  dddc8d12ac1      1
3  bcfe8f51667   

## Intento 4

In [15]:
import pandas as pd
import numpy as np
import string
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Input, Dropout, GlobalAveragePooling1D, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime

# Cargar el modelo de spaCy
nlp = spacy.load("en_core_web_sm")


# Tokenización
max_vocab = 10000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(df['processed_statement'])

# Convertir los textos en secuencias de tokens
X_train = tokenizer.texts_to_sequences(df['processed_statement'])
X_test = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de las secuencias
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encoding de las columnas categóricas adicionales
subject_encoded = pd.get_dummies(df['processed_subject'])
speaker_encoded = pd.get_dummies(df['speaker'])
party_affiliation_encoded = pd.get_dummies(df['party_affiliation'])

# Unir las características codificadas con las secuencias de texto
X_additional_features = pd.concat([subject_encoded, speaker_encoded, party_affiliation_encoded], axis=1)

# Labels
y = df['label'].values

# Dividir el dataset en entrenamiento y validación (80-20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# Calcular los pesos de clase para balancear el dataset
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Definir las entradas para el modelo
input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
input_additional = Input(shape=(X_additional_features.shape[1],))  # Características adicionales

# Capa de embedding para el texto
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_text)

# Capa LSTM bidireccional con más unidades y Dropout
lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(embedding_layer)

# Más unidades LSTM
dropout_layer = Dropout(0.5)(lstm_layer)

# Capa de atención
attention_layer = Attention()([dropout_layer, dropout_layer])

# Capa de pooling
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Concatenar las características adicionales
merged = Concatenate()([pooling_layer, input_additional])

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(merged)

# Definir el modelo
model = Model(inputs=[input_text, input_additional], outputs=output_layer)

# Crear un callback para reducir la tasa de aprendizaje si el modelo no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo con el callback de ReduceLROnPlateau
model.fit([X_train, X_additional_features.iloc[:len(X_train)]], y_train, epochs=5, batch_size=32,
          validation_data=([X_val, X_additional_features.iloc[:len(X_val)]], y_val),
          class_weight={0: class_weights[0], 1: class_weights[1]},
          callbacks=[reduce_lr])

# Predicción sobre el conjunto de prueba
y_pred_prob = model.predict([X_test, X_additional_features.iloc[:len(X_test)]])
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades en etiquetas binarias (0 o 1)

# Generar el CSV con las predicciones para df_test
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que 'y_pred' tenga la forma correcta
})

# Guardar el archivo de predicciones
filename = f"biLSTM_submission_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

# Mensaje final
print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())


c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 100, 100)  │  1,000,000 │ input_layer_10[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_5     │ (None, 100, 512)  │    731,136 │ embedding_5[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 100, 512)  │          0 │ bidirectional_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_5         │ (None, 100, 512)  │          0 │ dropout_5[0][0],  │
│ (Attention)         │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ attention_5[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_11      │ (None, 6067)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 6579)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ input_layer_11[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │      6,580 │ concatenate_5[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,737,716 (6.63 MB)

 Trainable params: 1,737,716 (6.63 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 39s 163ms/step - accuracy: 0.5128 - loss: 0.6949 - val_accuracy: 0.4385 - val_loss: 0.6959 - learning_rate: 0.0010
Epoch 2/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 30s 134ms/step - accuracy: 0.5030 - loss: 0.6949 - val_accuracy: 0.6235 - val_loss: 0.6854 - learning_rate: 0.0010
Epoch 3/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 33s 147ms/step - accuracy: 0.7112 - loss: 0.6808 - val_accuracy: 0.4989 - val_loss: 0.6933 - learning_rate: 0.0010
Epoch 4/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 32s 144ms/step - accuracy: 0.7169 - loss: 0.6728 - val_accuracy: 0.5676 - val_loss: 0.6898 - learning_rate: 0.0010
Epoch 5/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 31s 140ms/step - accuracy: 0.6460 - loss: 0.6306 - val_accuracy: 0.5765 - val_loss: 0.7132 - learning_rate: 5.0000e-04
120/120 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step
✅ Submission generada correctamente: 'biLSTM_submission_20250514_0253.csv'
            id  label
0  dc32e5ffa8b      0
1  aa49bb41cab      0
2  dddc8d12ac1      1
3  bcfe8f51667      1

In [ ]:
import pandas as pd
import numpy as np
import string
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Input, Dropout, GlobalAveragePooling1D, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime




# Tokenización
max_vocab = 10000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(df['processed_statement'])

# Convertir los textos en secuencias de tokens
X_train = tokenizer.texts_to_sequences(df['processed_statement'])
X_test = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de las secuencias
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encoding de las columnas categóricas adicionales
subject_encoded = pd.get_dummies(df['processed_subject'])
speaker_encoded = pd.get_dummies(df['speaker_job'])
party_affiliation_encoded = pd.get_dummies(df['party_affiliation_uni'])

# Unir las características codificadas con las secuencias de texto
X_additional_features = pd.concat([subject_encoded, speaker_encoded, party_affiliation_encoded], axis=1)

# Labels
y = df['label'].values

# Dividir el dataset en entrenamiento y validación (80-20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# Calcular los pesos de clase para balancear el dataset
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Definir las entradas para el modelo
input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
input_additional = Input(shape=(X_additional_features.shape[1],))  # Características adicionales

# Capa de embedding para el texto
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_text)

# Capa LSTM bidireccional con más unidades y Dropout
lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(embedding_layer)

# Más unidades LSTM y dropout para regularización
dropout_layer = Dropout(0.5)(lstm_layer)

# Capa de atención
attention_layer = Attention()([dropout_layer, dropout_layer])

# Capa de pooling global
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Concatenar las características adicionales
merged = Concatenate()([pooling_layer, input_additional])

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(merged)

# Definir el modelo
model = Model(inputs=[input_text, input_additional], outputs=output_layer)

# Crear un callback para reducir la tasa de aprendizaje si el modelo no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo con el callback de ReduceLROnPlateau
model.fit([X_train, X_additional_features.iloc[:len(X_train)]], y_train, epochs=5, batch_size=32,
          validation_data=([X_val, X_additional_features.iloc[:len(X_val)]], y_val),
          class_weight={0: class_weights[0], 1: class_weights[1]},
          callbacks=[reduce_lr])

# Predicción sobre el conjunto de prueba
y_pred_prob = model.predict([X_test, X_additional_features.iloc[:len(X_test)]])
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades en etiquetas binarias (0 o 1)

# Generar el CSV con las predicciones para df_test
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que 'y_pred' tenga la forma correcta
})

# Guardar el archivo de predicciones
filename = f"biLSTM_submission_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

# Mensaje final
print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())


c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, 100, 100)  │  1,000,000 │ input_layer_14[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_7     │ (None, 100, 512)  │    731,136 │ embedding_7[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 100, 512)  │          0 │ bidirectional_7[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_7         │ (None, 100, 512)  │          0 │ dropout_7[0][0],  │
│ (Attention)         │                   │            │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ attention_7[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, 4498)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_7       │ (None, 5010)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ input_layer_15[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 1)         │      5,011 │ concatenate_7[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,736,147 (6.62 MB)

 Trainable params: 1,736,147 (6.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 70s 302ms/step - accuracy: 0.5095 - loss: 0.6921 - val_accuracy: 0.4486 - val_loss: 0.6983 - learning_rate: 0.0010
Epoch 2/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 70s 310ms/step - accuracy: 0.5266 - loss: 0.6916 - val_accuracy: 0.5542 - val_loss: 0.6891 - learning_rate: 0.0010
Epoch 3/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 67s 299ms/step - accuracy: 0.6371 - loss: 0.6831 - val_accuracy: 0.6408 - val_loss: 0.6762 - learning_rate: 0.0010
Epoch 4/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 67s 298ms/step - accuracy: 0.6153 - loss: 0.6622 - val_accuracy: 0.6575 - val_loss: 0.7355 - learning_rate: 0.0010
Epoch 5/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 66s 295ms/step - accuracy: 0.6284 - loss: 0.6357 - val_accuracy: 0.6089 - val_loss: 0.7378 - learning_rate: 0.0010
120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step
✅ Submission generada correctamente: 'biLSTM_submission_20250514_0305.csv'
            id  label
0  dc32e5ffa8b      1
1  aa49bb41cab      0
2  dddc8d12ac1      1
3  bcfe8f51667      1
4 

In [19]:
import pandas as pd
import numpy as np
import string
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Input, Dropout, GlobalAveragePooling1D, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime




# Tokenización
max_vocab = 10000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(df['processed_statement'])

# Convertir los textos en secuencias de tokens
X_train = tokenizer.texts_to_sequences(df['processed_statement'])
X_test = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de las secuencias
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encoding de las columnas categóricas adicionales
subject_encoded = pd.get_dummies(df['processed_subject'])
speaker_encoded = pd.get_dummies(df['speaker_job'])
party_affiliation_encoded = pd.get_dummies(df['party_affiliation_category_map'])

# Unir las características codificadas con las secuencias de texto
X_additional_features = pd.concat([subject_encoded, speaker_encoded, party_affiliation_encoded], axis=1)

# Labels
y = df['label'].values

# Dividir el dataset en entrenamiento y validación (80-20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# Calcular los pesos de clase para balancear el dataset
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Definir las entradas para el modelo
input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
input_additional = Input(shape=(X_additional_features.shape[1],))  # Características adicionales

# Capa de embedding para el texto
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_text)

# Capa LSTM bidireccional con más unidades y Dropout
lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(embedding_layer)

# Más unidades LSTM y dropout para regularización
dropout_layer = Dropout(0.5)(lstm_layer)

# Capa de atención
attention_layer = Attention()([dropout_layer, dropout_layer])

# Capa de pooling global
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Concatenar las características adicionales
merged = Concatenate()([pooling_layer, input_additional])

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(merged)

# Definir el modelo
model = Model(inputs=[input_text, input_additional], outputs=output_layer)

# Crear un callback para reducir la tasa de aprendizaje si el modelo no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo con el callback de ReduceLROnPlateau
model.fit([X_train, X_additional_features.iloc[:len(X_train)]], y_train, epochs=5, batch_size=32,
          validation_data=([X_val, X_additional_features.iloc[:len(X_val)]], y_val),
          class_weight={0: class_weights[0], 1: class_weights[1]},
          callbacks=[reduce_lr])

# Predicción sobre el conjunto de prueba
y_pred_prob = model.predict([X_test, X_additional_features.iloc[:len(X_test)]])
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades en etiquetas binarias (0 o 1)

# Generar el CSV con las predicciones para df_test
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que 'y_pred' tenga la forma correcta
})

# Guardar el archivo de predicciones
filename = f"biLSTM_submission_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

# Mensaje final
print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())


c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, 100, 100)  │  1,000,000 │ input_layer_18[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_9     │ (None, 100, 512)  │    731,136 │ embedding_9[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 100, 512)  │          0 │ bidirectional_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_9         │ (None, 100, 512)  │          0 │ dropout_9[0][0],  │
│ (Attention)         │                   │            │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ attention_9[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, 4440)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_9       │ (None, 4952)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ input_layer_19[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │      4,953 │ concatenate_9[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,736,089 (6.62 MB)

 Trainable params: 1,736,089 (6.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 37s 156ms/step - accuracy: 0.5418 - loss: 0.6912 - val_accuracy: 0.4391 - val_loss: 0.6964 - learning_rate: 0.0010
Epoch 2/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 34s 152ms/step - accuracy: 0.5490 - loss: 0.6876 - val_accuracy: 0.5598 - val_loss: 0.6894 - learning_rate: 0.0010
Epoch 3/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 35s 158ms/step - accuracy: 0.6311 - loss: 0.6849 - val_accuracy: 0.6106 - val_loss: 0.6847 - learning_rate: 0.0010
Epoch 4/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 31s 138ms/step - accuracy: 0.6683 - loss: 0.6727 - val_accuracy: 0.5145 - val_loss: 0.7110 - learning_rate: 0.0010
Epoch 5/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 30s 135ms/step - accuracy: 0.6598 - loss: 0.6460 - val_accuracy: 0.5732 - val_loss: 0.7015 - learning_rate: 0.0010
120/120 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step
✅ Submission generada correctamente: 'biLSTM_submission_20250514_0310.csv'
            id  label
0  dc32e5ffa8b      0
1  aa49bb41cab      0
2  dddc8d12ac1      1
3  bcfe8f51667      1
4  

In [20]:
import pandas as pd
import numpy as np
import string
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Input, Dropout, GlobalAveragePooling1D, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime




max_vocab = 20000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(df['processed_statement'])

# Convertir los textos en secuencias de tokens
X_train = tokenizer.texts_to_sequences(df['processed_statement'])
X_test = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de las secuencias
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encoding de las columnas categóricas adicionales
subject_encoded = pd.get_dummies(df['processed_subject'])
speaker_encoded = pd.get_dummies(df['speaker_job'])
party_affiliation_encoded = pd.get_dummies(df['party_affiliation_uni'])

# Unir las características codificadas con las secuencias de texto
X_additional_features = pd.concat([subject_encoded, speaker_encoded, party_affiliation_encoded], axis=1)

# Labels
y = df['label'].values

# Dividir el dataset en entrenamiento y validación (80-20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# Calcular los pesos de clase para balancear el dataset
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Definir las entradas para el modelo
input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
input_additional = Input(shape=(X_additional_features.shape[1],))  # Características adicionales

# Capa de embedding para el texto
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_text)

# Capa LSTM bidireccional con más unidades y Dropout
lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(embedding_layer)

# Más unidades LSTM y dropout para regularización
dropout_layer = Dropout(0.5)(lstm_layer)

# Capa de atención
attention_layer = Attention()([dropout_layer, dropout_layer])

# Capa de pooling global
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Concatenar las características adicionales
merged = Concatenate()([pooling_layer, input_additional])

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(merged)

# Definir el modelo
model = Model(inputs=[input_text, input_additional], outputs=output_layer)

# Crear un callback para reducir la tasa de aprendizaje si el modelo no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo con el callback de ReduceLROnPlateau
model.fit([X_train, X_additional_features.iloc[:len(X_train)]], y_train, epochs=5, batch_size=32,
          validation_data=([X_val, X_additional_features.iloc[:len(X_val)]], y_val),
          class_weight={0: class_weights[0], 1: class_weights[1]},
          callbacks=[reduce_lr])

# Predicción sobre el conjunto de prueba
y_pred_prob = model.predict([X_test, X_additional_features.iloc[:len(X_test)]])
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades en etiquetas binarias (0 o 1)

# Generar el CSV con las predicciones para df_test
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que 'y_pred' tenga la forma correcta
})

# Guardar el archivo de predicciones
filename = f"biLSTM_submission_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

# Mensaje final
print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())


c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, 100, 100)  │  2,000,000 │ input_layer_20[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_10    │ (None, 100, 512)  │    731,136 │ embedding_10[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 100, 512)  │          0 │ bidirectional_10… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_10        │ (None, 100, 512)  │          0 │ dropout_10[0][0], │
│ (Attention)         │                   │            │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ attention_10[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_21      │ (None, 4448)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_10      │ (None, 4960)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ input_layer_21[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 1)         │      4,961 │ concatenate_10[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,736,097 (10.44 MB)

 Trainable params: 2,736,097 (10.44 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 37s 153ms/step - accuracy: 0.4389 - loss: 0.6961 - val_accuracy: 0.4950 - val_loss: 0.6936 - learning_rate: 0.0010
Epoch 2/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 32s 143ms/step - accuracy: 0.5725 - loss: 0.6882 - val_accuracy: 0.4804 - val_loss: 0.6942 - learning_rate: 0.0010
Epoch 3/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 31s 138ms/step - accuracy: 0.6083 - loss: 0.6825 - val_accuracy: 0.5246 - val_loss: 0.6857 - learning_rate: 0.0010
Epoch 4/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 32s 142ms/step - accuracy: 0.6365 - loss: 0.6400 - val_accuracy: 0.5961 - val_loss: 0.6933 - learning_rate: 0.0010
Epoch 5/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 26s 115ms/step - accuracy: 0.7514 - loss: 0.4962 - val_accuracy: 0.5855 - val_loss: 0.7915 - learning_rate: 0.0010
120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step
✅ Submission generada correctamente: 'biLSTM_submission_20250514_0313.csv'
            id  label
0  dc32e5ffa8b      0
1  aa49bb41cab      0
2  dddc8d12ac1      1
3  bcfe8f51667      1
4  

In [21]:
import pandas as pd
import numpy as np
import string
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Input, Dropout, GlobalAveragePooling1D, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime




# Tokenización
max_vocab = 10000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=max_vocab, oov_token="<OOV>")
tokenizer.fit_on_texts(df['processed_statement'])

# Convertir los textos en secuencias de tokens
X_train = tokenizer.texts_to_sequences(df['processed_statement'])
X_test = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de las secuencias
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encoding de las columnas categóricas adicionales
subject_encoded = pd.get_dummies(df['processed_subject'])
speaker_encoded = pd.get_dummies(df['speaker_job'])
party_affiliation_encoded = pd.get_dummies(df['party_affiliation_uni'])

# Unir las características codificadas con las secuencias de texto
X_additional_features = pd.concat([subject_encoded, speaker_encoded, party_affiliation_encoded], axis=1)

# Labels
y = df['label'].values

# Dividir el dataset en entrenamiento y validación (80-20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# Calcular los pesos de clase para balancear el dataset
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Definir las entradas para el modelo
input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
input_additional = Input(shape=(X_additional_features.shape[1],))  # Características adicionales

# Capa de embedding para el texto
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_text)

# Capa LSTM bidireccional con más unidades y Dropout
lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(embedding_layer)

# Más unidades LSTM y dropout para regularización
dropout_layer = Dropout(0.5)(lstm_layer)

# Capa de atención
attention_layer = Attention()([dropout_layer, dropout_layer])

# Capa de pooling global
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Concatenar las características adicionales
merged = Concatenate()([pooling_layer, input_additional])

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(merged)

# Definir el modelo
model = Model(inputs=[input_text, input_additional], outputs=output_layer)

# Crear un callback para reducir la tasa de aprendizaje si el modelo no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo con el callback de ReduceLROnPlateau
model.fit([X_train, X_additional_features.iloc[:len(X_train)]], y_train, epochs=5, batch_size=32,
          validation_data=([X_val, X_additional_features.iloc[:len(X_val)]], y_val),
          class_weight={0: class_weights[0], 1: class_weights[1]},
          callbacks=[reduce_lr])

# Predicción sobre el conjunto de prueba
y_pred_prob = model.predict([X_test, X_additional_features.iloc[:len(X_test)]])
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades en etiquetas binarias (0 o 1)

# Generar el CSV con las predicciones para df_test
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que 'y_pred' tenga la forma correcta
})

# Guardar el archivo de predicciones
filename = f"biLSTM_submission_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

# Mensaje final
print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())


c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, 100, 100)  │  1,000,000 │ input_layer_22[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_11    │ (None, 100, 512)  │    731,136 │ embedding_11[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 100, 512)  │          0 │ bidirectional_11… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_11        │ (None, 100, 512)  │          0 │ dropout_11[0][0], │
│ (Attention)         │                   │            │ dropout_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ attention_11[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_23      │ (None, 4448)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_11      │ (None, 4960)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ input_layer_23[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │      4,961 │ concatenate_11[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,736,097 (6.62 MB)

 Trainable params: 1,736,097 (6.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 30s 122ms/step - accuracy: 0.5252 - loss: 0.6932 - val_accuracy: 0.5106 - val_loss: 0.6926 - learning_rate: 0.0010
Epoch 2/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 25s 113ms/step - accuracy: 0.5653 - loss: 0.6946 - val_accuracy: 0.5978 - val_loss: 0.6872 - learning_rate: 0.0010
Epoch 3/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 26s 116ms/step - accuracy: 0.6489 - loss: 0.6832 - val_accuracy: 0.5687 - val_loss: 0.7083 - learning_rate: 0.0010
Epoch 4/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 26s 116ms/step - accuracy: 0.6257 - loss: 0.6495 - val_accuracy: 0.5145 - val_loss: 0.6845 - learning_rate: 0.0010
Epoch 5/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 27s 118ms/step - accuracy: 0.6059 - loss: 0.6444 - val_accuracy: 0.5788 - val_loss: 0.6841 - learning_rate: 0.0010
120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step
✅ Submission generada correctamente: 'biLSTM_submission_20250514_0316.csv'
            id  label
0  dc32e5ffa8b      0
1  aa49bb41cab      0
2  dddc8d12ac1      1
3  bcfe8f51667      1
4  

In [ ]:
import pandas as pd
import numpy as np
import string
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Input, Dropout, GlobalAveragePooling1D, Attention, Concatenate, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Nadam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from datetime import datetime

# Función para extraer n-gramas (bigramas o trigramas)
def get_ngrams(text, n=2):
    tokens = text.split()  # Dividir el texto en palabras
    ngrams = [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]  # Extraer n-gramas
    return ' '.join(ngrams)  # Unir los n-gramas en una cadena


# Aplicar n-gramas (bigramas en este caso) al conjunto de datos
df['processed_statement'] = df['processed_statement'].apply(lambda x: get_ngrams(x, n=2))  # Bigrams
df_test['processed_statement'] = df_test['processed_statement'].apply(lambda x: get_ngrams(x, n=2))  # Bigrams

# Inicializar el Tokenizer con OOV
max_vocab = 20000  # Tamaño máximo del vocabulario
tokenizer = Tokenizer(num_words=max_vocab, oov_token="<OOV>")
tokenizer.fit_on_texts(df['processed_statement'])  # Ajustar el Tokenizer a los n-gramas

# Convertir los textos procesados (con n-gramas) en secuencias de tokens
X_train = tokenizer.texts_to_sequences(df['processed_statement'])
X_test = tokenizer.texts_to_sequences(df_test['processed_statement'])

# Padding de las secuencias
MAX_SEQUENCE_LENGTH = 100
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encoding de las columnas categóricas adicionales
subject_encoded = pd.get_dummies(df['processed_subject'])
speaker_encoded = pd.get_dummies(df['speaker'])
party_affiliation_encoded = pd.get_dummies(df['party_affiliation'])

# Unir las características codificadas con las secuencias de texto
X_additional_features = pd.concat([subject_encoded, speaker_encoded, party_affiliation_encoded], axis=1)

# Labels
y = df['label'].values

# Dividir el dataset en entrenamiento y validación (80-20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# Calcular los pesos de clase para balancear el dataset
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Definir las entradas para el modelo
input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
input_additional = Input(shape=(X_additional_features.shape[1],))  # Características adicionales

# Capa de embedding para el texto
embedding_layer = Embedding(input_dim=max_vocab, output_dim=100, input_length=MAX_SEQUENCE_LENGTH)(input_text)

# Aumentar la complejidad del modelo con más capas LSTM y atención multi-cabeza
lstm_layer = Bidirectional(LSTM(512, return_sequences=True))(embedding_layer)
dropout_layer = Dropout(0.5)(lstm_layer)

# Implementar Multi-Head Attention
attention_layer = MultiHeadAttention(num_heads=8, key_dim=256)(dropout_layer, dropout_layer)

# Capa de pooling global
pooling_layer = GlobalAveragePooling1D()(attention_layer)

# Añadir una capa densa intermedia para mejorar el modelado no lineal
dense_layer = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(pooling_layer)

# Concatenar las características adicionales
merged = Concatenate()([dense_layer, input_additional])

# Capa de salida
output_layer = Dense(1, activation='sigmoid')(merged)

# Definir el modelo
model = Model(inputs=[input_text, input_additional], outputs=output_layer)

# Crear un callback para reducir la tasa de aprendizaje si el modelo no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Definir un scheduler para la tasa de aprendizaje
def lr_schedule(epoch):
    if epoch < 10:
        return 1e-3
    elif epoch < 20:
        return 1e-4
    else:
        return 1e-5

lr_scheduler = LearningRateScheduler(lr_schedule)

# Compilar el modelo con el optimizador Nadam
model.compile(optimizer=Nadam(), loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo con el callback de ReduceLROnPlateau
model.fit([X_train, X_additional_features.iloc[:len(X_train)]], y_train, epochs=5, batch_size=32,
          validation_data=([X_val, X_additional_features.iloc[:len(X_val)]], y_val),
          class_weight={0: class_weights[0], 1: class_weights[1]},
          callbacks=[reduce_lr, lr_scheduler])

# Predicción sobre el conjunto de prueba
y_pred_prob = model.predict([X_test, X_additional_features.iloc[:len(X_test)]])
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades en etiquetas binarias (0 o 1)

# Generar el CSV con las predicciones para df_test
submission = pd.DataFrame({
    'id': df_test['id'],
    'label': y_pred.flatten()  # Asegurarse de que 'y_pred' tenga la forma correcta
})

# Guardar el archivo de predicciones
filename = f"biLSTM_submission_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
submission.to_csv(filename, columns=['id', 'label'], index=False)

# Mensaje final
print(f"✅ Submission generada correctamente: '{filename}'")
print(submission.head())

c:\Users\anaeg\Desktop\LBBYs_CH2\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, 100, 100)  │  2,000,000 │ input_layer_24[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_12    │ (None, 100, 1024) │  2,510,848 │ embedding_12[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 100, 1024) │          0 │ bidirectional_12… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 1024) │  8,395,776 │ dropout_12[0][0], │
│ (MultiHeadAttentio… │                   │            │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1024)      │          0 │ multi_head_atten… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 128)       │    131,200 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_25      │ (None, 6067)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_12      │ (None, 6195)      │          0 │ dense_12[0][0],   │
│ (Concatenate)       │                   │            │ input_layer_25[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 1)         │      6,196 │ concatenate_12[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,044,020 (49.76 MB)

 Trainable params: 13,044,020 (49.76 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
224/224 ━━━━━━━━━━━━━━━━━━━━ 186s 814ms/step - accuracy: 0.5275 - loss: 0.7782 - val_accuracy: 0.4430 - val_loss: 0.7005 - learning_rate: 0.0010
Epoch 2/5
 15/224 ━━━━━━━━━━━━━━━━━━━━ 2:42 775ms/step - accuracy: 0.6355 - loss: 0.6753